TO-DO:
- add merge_geom parameter to dissolve to automatically pull that using the created relation
- add cut and interpolate aggregators

In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
from shapely.geometry import LineString
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
lr.set_default_lrs(key_col=['county', 'route'], loc_col='loc', beg_col='beg', end_col='end', geom_col='geometry', closed='left_mod')

df = gpd.read_file('testing/data/linear-events-3.geojson').set_index('index')
df.lr

LRS_Accessor with linear referencing system (LRS) objects:
- LRS(key_col=['county', 'route'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, geom_m_col=None, closed=left_mod)

In [3]:
df.lr.add_chaining(inplace=True)
df.lr

LRS_Accessor with linear referencing system (LRS) objects:
- LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, geom_m_col=None, closed=left_mod)

In [4]:
df.lr.add_geom_m(inplace=True)
df.lr

LRS_Accessor with linear referencing system (LRS) objects:
- LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, geom_m_col=None, closed=left_mod)
- LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, geom_m_col=geometry_m, closed=left_mod)

In [5]:
df

,county,route,beg,end,volume,class,geometry,chain,geometry_m
index,,,,,,,,,
7,A,Main St,0.0,0.4,100,B,"LINESTRING (0 0, 0.8 0)",0,"LINESTRING M (0.0 0.0 0.0, 0.8 0.0 0.4) # linr..."
6,A,Main St,0.5,1.1,200,A,"LINESTRING (1 0, 1 1)",1,"LINESTRING M (1.0 0.0 0.5, 1.0 1.0 1.1) # linr..."
9,A,Main St,1.1,3.0,200,A,"LINESTRING (1 1, 2 1)",1,"LINESTRING M (1.0 1.0 1.1, 2.0 1.0 3.0) # linr..."
4,A,1st Ave,10.0,12.0,400,A,"LINESTRING (10 0, 12 0)",0,"LINESTRING M (10.0 0.0 10.0, 12.0 0.0 12.0) # ..."
3,A,1st Ave,12.0,13.0,500,B,"LINESTRING (12 0, 13 0)",0,"LINESTRING M (12.0 0.0 12.0, 13.0 0.0 13.0) # ..."
2,B,2nd Ave,13.0,13.1,150,A,"LINESTRING (13 0, 13 1)",0,"LINESTRING M (13.0 0.0 13.0, 13.0 1.0 13.1) # ..."
0,B,2nd Ave,13.1,14.0,200,A,"LINESTRING (13 1, 14 1)",0,"LINESTRING M (13.0 1.0 13.1, 14.0 1.0 14.0) # ..."
1,B,2nd Ave,14.0,17.0,300,B,"LINESTRING (14 1, 17 1)",0,"LINESTRING M (14.0 1.0 14.0, 17.0 1.0 17.0) # ..."
10,B,2nd Ave,17.0,17.8,200,A,"LINESTRING (17 1, 17 2)",0,"LINESTRING M (17.0 1.0 17.0, 17.0 2.0 17.8) # ..."


In [6]:
dissolved, relation = df.lr.dissolve(return_relation=True)

In [7]:
dissolved

,county,route,chain,beg,end,dissolved_index,geometry,geometry_m
index,,,,,,,,
0,A,Main St,0,0.0,0.4,[7],"LINESTRING (0 0, 0.8 0)","LINESTRING M (0.0 0.0 0.0, 0.8 0.0 0.4) # linr..."
1,A,Main St,1,0.5,3.0,"[6, 9]","LINESTRING (1 0, 1 1, 2 1)","LINESTRING M (1.0 0.0 0.5, 1.0 1.0 1.1, 2.0 1...."
2,A,1st Ave,0,10.0,13.0,"[4, 3]","LINESTRING (10 0, 12 0, 13 0)","LINESTRING M (10.0 0.0 10.0, 12.0 0.0 12.0, 13..."
3,B,2nd Ave,0,13.0,18.1,"[2, 0, 1, 10, 13]","LINESTRING (13 0, 13 1, 14 1, 17 1, 17 2, 17 3)","LINESTRING M (13.0 0.0 13.0, 13.0 1.0 13.1, 14..."
4,C,Sycamore School Rd,0,1.8,3.0,[8],"LINESTRING (1 3, 1 4)","LINESTRING M (1.0 3.0 1.8, 1.0 4.0 3.0) # linr..."


In [7]:
relation['geometry_m'].linemerge_m()

0    LINESTRING M (0.0 0.0 0.0, 0.8 0.0 0.4) # linr...
1    LINESTRING M (1.0 0.0 0.5, 1.0 1.0 1.1, 2.0 1....
2    LINESTRING M (10.0 0.0 10.0, 12.0 0.0 12.0, 13...
3    LINESTRING M (13.0 0.0 13.0, 13.0 1.0 13.1, 14...
4    LINESTRING M (1.0 3.0 1.8, 1.0 4.0 3.0) # linr...
dtype: object

In [12]:
relation[['volume', 'class']].mode()

array([[100, 'B'],
       [200, 'A'],
       [400, 'A'],
       [300, 'B'],
       [800, 'A']], dtype=object)